In [1]:
import tensorflow as tf
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = x_train.reshape(60000, 784)

scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)

class BFR:
    def __init__(self, K1):
        self.K1 = K1
        
    def fit(self, X):
        n, m = X.shape
        self.cluster_centers_ = np.zeros((10, m))
        self.cluster_sizes_ = np.zeros(10)
        self.cluster_labels_ = np.zeros(n, dtype=int)
        self.buffer_ = np.zeros((self.K1, m))
        self.buffer_size_ = 0
        
        # perform initial clustering on a random subset of size K1
        kmeans = KMeans(n_clusters=10)
        sample_indices = np.random.choice(n, self.K1, replace=False)
        kmeans.fit(X[sample_indices])
        self.cluster_centers_ = kmeans.cluster_centers_
        
        # assign samples to the nearest cluster
        for i in range(n):
            if self.buffer_size_ == self.K1:
                # buffer is full, flush it to the nearest cluster
                self._flush_buffer()
            
            x = X[i]
            label = kmeans.predict(x.reshape(1, -1))[0]
            self.buffer_[self.buffer_size_] = x
            self.cluster_sizes_[label] += 1
            self.cluster_labels_[i] = label
            self.buffer_size_ += 1
            
        # flush remaining samples in buffer to the nearest cluster
        self._flush_buffer()
    
    def _flush_buffer(self):
        kmeans = KMeans(n_clusters=10, init=self.cluster_centers_)
        kmeans.fit(self.buffer_[:self.buffer_size_])
        self.cluster_centers_ = kmeans.cluster_centers_
        
        for i in range(self.buffer_size_):
            label = kmeans.predict(self.buffer_[i].reshape(1, -1))[0]
            self.cluster_sizes_[label] += 1
            self.cluster_labels_[i] = label
        
        self.buffer_size_ = 0


bfr = BFR(K1=100)
bfr.fit(x_train)

# calculate the percentage of samples in each cluster
cluster_sizes = bfr.cluster_sizes_ / x_train.shape[0]
print(cluster_sizes)


11490434/11490434 [==============================] - 0s 0us/step


/usr/local/lib/python3.8/dist-packages/sklearn/cluster/_kmeans.py:1146: RuntimeWarning: Explicit initial center position passed: performing only one init in KMeans instead of n_init=10.
  self._check_params(X)
/usr/local/lib/python3.8/dist-packages/sklearn/cluster/_kmeans.py:1146: RuntimeWarning: Explicit initial center position passed: performing only one init in KMeans instead of n_init=10.
  self._check_params(X)
/usr/local/lib/python3.8/dist-packages/sklearn/cluster/_kmeans.py:1146: RuntimeWarning: Explicit initial center position passed: performing only one init in KMeans instead of n_init=10.
  self._check_params(X)
/usr/local/lib/python3.8/dist-packages/sklearn/cluster/_kmeans.py:1146: RuntimeWarning: Explicit initial center position passed: performing only one init in KMeans instead of n_init=10.
  self._check_params(X)
/usr/local/lib/python3.8/dist-packages/sklearn/cluster/_kmeans.py:1146: RuntimeWarning: Explicit initial center position passed: performing only one init in KMe

[0.1264     0.06683333 0.07453333 0.15633333 0.62821667 0.2816
 0.35216667 0.16856667 0.08375    0.0616    ]


In [2]:
# calculate the entropy of each cluster
cluster_entropies = [0] * 10
for i in range(10):
    indices = np.where(bfr.cluster_labels_ == i)[0]
    if len(indices) > 0:
        classes, counts = np.unique(y_train[indices], return_counts=True)
        proportion = counts / len(indices)
        entropy = -np.sum(proportion * np.log2(proportion))
        cluster_entropies[i] = entropy
print(cluster_entropies)

# calculate the total entropy of all clusters
total_entropy = np.sum(cluster_sizes * cluster_entropies)
print(total_entropy)

[1.869091049640708, 1.9362600275315274, 3.2102887819669936, 1.0118920306556365, 3.023470641386182, 3.026244431571033, 2.6196184786426304, 0.8765273284219849, 2.0515674922257645, 1.2952582108395383]
4.836613159367541


In [6]:
import numpy as np

def calculate_entropy(cluster_assignments, y_train, num_clusters):
    cluster_entropies = []
    total_entropy = 0
    for i in range(num_clusters):
        indices = np.where(cluster_assignments == i)[0]
        cluster_size = len(indices)
        if cluster_size == 0:
            continue
        labels = y_train[indices]
        unique_labels, label_counts = np.unique(labels, return_counts=True)
        cluster_entropy = 0
        for count in label_counts:
            p = count / cluster_size
            cluster_entropy -= p * np.log2(p)
        cluster_entropies.append(cluster_entropy)
        total_entropy += (cluster_size / len(y_train)) * cluster_entropy
    return np.array(cluster_entropies), total_entropy

In [7]:
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist


def run_bfr_multiple_times(X_train, y_train, k1_list, num_runs=5):
    """
    Run BFR algorithm multiple times with different values of k1 and return results.

    Args:
        X_train (numpy.ndarray): Training data features.
        y_train (numpy.ndarray): Training data labels.
        k1_list (list): List of values of k1 to try.
        num_runs (int, optional): Number of runs for each k1 value. Defaults to 5.

    Returns:
        dict: Dictionary containing the results of the BFR algorithm for each k1 value.
              The keys are the k1 values, and the values are dictionaries containing
              the cluster entropies and total entropy.
    """
    results = {}
    for k1 in k1_list:
        cluster_entropies = []
        total_entropy = 0
        for i in range(num_runs):
            # Initialize cluster centers using KMeans
            kmeans = KMeans(n_clusters=10, random_state=0).fit(X_train[:k1])
            cluster_centers = kmeans.cluster_centers_

            # Initialize buffer and assign samples to clusters
            buffer = []
            cluster_assignments = np.zeros(len(X_train), dtype=int)
            for j, x in enumerate(X_train):
                if len(buffer) < k1:
                    buffer.append(x)
                    continue
                distances = cdist([x], cluster_centers)
                nearest_cluster = np.argmin(distances)
                if len(buffer) > k1:
                    # Flush buffer to nearest cluster
                    for b in buffer:
                        cluster_assignments[j - k1] = nearest_cluster
                        distances = cdist([b], cluster_centers)
                        nearest_cluster = np.argmin(distances)
                    buffer = []
                buffer.append(x)
                cluster_assignments[j] = nearest_cluster

            # Calculate entropy of each cluster and total entropy
            cluster_entropies_i, total_entropy_i = calculate_entropy(
                cluster_assignments, y_train, 10)
            cluster_entropies.append(cluster_entropies_i)
            total_entropy += total_entropy_i

        # Save the results for this k1 value
        results[k1] = {
            'cluster_entropies': np.array(cluster_entropies),
            'total_entropy': total_entropy / num_runs
        }

    return results


# Define the list of k1 values to try
k1_list = [100, 200, 500]

# Run BFR algorithm multiple times with different k1 values
bfr_results = run_bfr_multiple_times(x_train, y_train, k1_list)

# Print the results
for k1 in k1_list:
    print(f"Results for k1={k1}:")
    print("Cluster entropies:", bfr_results[k1]['cluster_entropies'])
    print("Total entropy:", bfr_results[k1]['total_entropy'])


Results for k1=100:
Cluster entropies: [[3.31995663 3.23232717 3.10535451 3.20382452 3.04197601 0.
  3.25230539 2.05881389]
 [3.31995663 3.23232717 3.10535451 3.20382452 3.04197601 0.
  3.25230539 2.05881389]
 [3.31995663 3.23232717 3.10535451 3.20382452 3.04197601 0.
  3.25230539 2.05881389]
 [3.31995663 3.23232717 3.10535451 3.20382452 3.04197601 0.
  3.25230539 2.05881389]
 [3.31995663 3.23232717 3.10535451 3.20382452 3.04197601 0.
  3.25230539 2.05881389]]
Total entropy: 3.3163279552379605
Results for k1=200:
Cluster entropies: [[3.31985503 3.21318892 3.21038695 3.26540929 2.82381205 2.91515123
  0.        ]
 [3.31985503 3.21318892 3.21038695 3.26540929 2.82381205 2.91515123
  0.        ]
 [3.31985503 3.21318892 3.21038695 3.26540929 2.82381205 2.91515123
  0.        ]
 [3.31985503 3.21318892 3.21038695 3.26540929 2.82381205 2.91515123
  0.        ]
 [3.31985503 3.21318892 3.21038695 3.26540929 2.82381205 2.91515123
  0.        ]]
Total entropy: 3.317887597356601
Results for k1=500